## L-test

In [1]:
from pyomo.environ import *
from pyomo.core.expr.calculus.derivatives import differentiate
from src.models.cz_model import CaoZavalaModel, CaoZavalaAlgo
from src.analyses.convergence_analysis import HausdorffAnalyzer
import dill

# from NSPLIB.src.instances.process.process_SP import const_model  #infeasible direction d lemma 5 failed
# sto_m = const_model()
# m = CaoZavalaModel.from_sto_m(sto_m)
# m.build()

# from NSPLIB.src.instances.ex8_4_4.ex8_4_4_SP import const_model   #0 purely equality constraints Proporisiton 2 in Kannan paper failed
# sto_m = const_model()
# m = CaoZavalaModel.from_sto_m(sto_m)
# m.build()

# from NSPLIB.src.instances.pooling_contract_selection.pooling import const_model #0.7 satisfies the conditions of Proposition 2 in Kannan paper
# sto_m = const_model()
# m = CaoZavalaModel.from_sto_m(sto_m)
# m.build()
# binary_ys = ['lambd[1]', 'lambd[2]', 'lambd[3]', 'lambd[4]', 'lambd[5]', 'theta[1]', 'theta[2]', 'theta[3]', 'theta[4]']

# binary_y_val = {y: 0 for y in binary_ys}
# binary_y_val['lambd[1]'] = 1
# binary_y_val['lambd[2]'] = 1
# binary_y_val['lambd[5]'] = 1
# binary_y_val['theta[1]'] = 1
# binary_y_val['theta[4]'] = 1
# m.fix_binary_y(binary_y_val)


from NSPLIB.src.instances.crude_selection.crude_selection import const_model #infeasible direction d lemma 5 failed
sto_m = const_model()
m = CaoZavalaModel.from_sto_m(sto_m)
m.build()
binary_ys = [f'pickCrude[{i}]' for i in range(1, 10 + 1)]
binary_y_val = {y: 1 for y in binary_ys}
# binary_y_val['pickCrude[2]'] = 1
# binary_y_val['pickCrude[3]'] = 1
# binary_y_val['pickCrude[4]'] = 1
# binary_y_val['pickCrude[8]'] = 1
# binary_y_val['pickCrude[10]'] = 1
m.fix_binary_y(binary_y_val)
# updated_y_bound=m.y_bound
# updated_y_bound['crudeQuantity[1]']=(0,0)
# updated_y_bound['crudeQuantity[5]']=(0,0)
# updated_y_bound['crudeQuantity[6]']=(0,0)
# updated_y_bound['crudeQuantity[7]']=(0,0)
# updated_y_bound['crudeQuantity[9]']=(0,0)
# m.update_y_bound(updated_y_bound)

from pyomo.opt import SolverFactory
solver = SolverFactory('baron')
#solver.options['AbsConFeasTol'] = 1e-12
solver.solve(m.origin_model,tee=True)


 BARON version 25.3.19. Built: OSX-64 2025-03-19 22:00:43         
 Running on machine Jingzhis-MacBook-Air.local

 BARON is a product of The Optimization Firm.
 For information on BARON, see https://minlp.com/about-baron
 Licensee: Jingzhi Yang at Georgia Institute of Technology, jyang872@gatech.edu.

 If you publish work using this software, please cite publications from
 https://minlp.com/baron-publications, such as: 

 Zhang, Y. and N. V. Sahinidis, Solving continuous and discrete
 nonlinear programs with BARON, Comput Optim Appl (2024). 
 https://doi.org/10.1007/s10589-024-00633-0
 This BARON run may utilize the following subsolver(s)
 For LP/MIP/QP: CLP/CBC                                         
 For NLP: FILTERSQP
 Doing local search
 Solving bounding LP
 Starting multi-start local search
 Preprocessing found feasible solution with value -17162.8
 Done with local search
  Iteration       Time (s)     Mem   Lower bound     Upper bound   Progress
          1           2.49    18

{'Problem': [{'Name': 'problem', 'Lower bound': -17162.829251366667, 'Upper bound': -17162.829251366667, 'Number of objectives': 1, 'Number of constraints': 577, 'Number of variables': 621, 'Sense': 'unknown', 'Missing bounds': '0', 'Iterations': '7', 'Node opt': '3', 'Node memmax': '3', 'Cpu time': 8.16, 'Wall time': 8.21}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Root node time': 2.49, 'Error rc': 0, 'Time': 8.2329580783844}], 'Solution': [OrderedDict({'number of solutions': 0, 'number of solutions displayed': 0})]}

In [2]:
from pyomo.core.expr.relational_expr import EqualityExpression, InequalityExpression
constraints = [c for c in m.origin_model.component_objects(Constraint, active=True)]
first_names=list(m.origin_model.y.keys())
equality_constraints = []
active_inequality_constraints = []
# if variable value is none set it to 0 to compute the Jacobian correctly
for v in m.origin_model.component_data_objects(Var, descend_into=True, active=True):
    if v.value is None:
        v.set_value(0)
# Collect equality and active inequality constraints into two dictionaries
for c in constraints:
    expr = c.expr
    if isinstance(expr, EqualityExpression):
        equality_constraints.append(c.name)
    elif isinstance(expr, InequalityExpression):
        if c.lower is None and abs(value(c)-value(c.upper))<1e-8:
            active_inequality_constraints.append(c.name)
        elif c.upper is None and abs(value(c)-value(c.lower))<1e-8:
            active_inequality_constraints.append(c.name)
        elif c.lower is not None and c.upper is not None:
            if abs(value(c)-value(c.lower))<1e-8 or abs(value(c)-value(c.upper))<1e-8:
                active_inequality_constraints.append(c.name)

jacobian_equality = {}
jacobian_inequality = {}
# Compute the Jacobian of equality constraints with respect to m.origin_model.y
for cname in equality_constraints:
    c_obj = m.origin_model.find_component(cname)
    jacobian_equality[cname] = []
    for vname in first_names:
        var = m.origin_model.y[vname]
        # For indexed constraints, differentiate each index
        expr = c_obj.expr.args[0]
        deriv = differentiate(expr, wrt=var)
        expr = c_obj.expr.args[1]
        deriv -= differentiate(expr, wrt=var)
        jacobian_equality[cname].append((vname, deriv))

# Compute the Jacobian of active inequality constraints with respect to m.origin_model.y
for cname in active_inequality_constraints:
    c_obj = m.origin_model.find_component(cname)
    jacobian_inequality[cname] = []
    for vname in first_names:
        var = m.origin_model.y[vname]
        # For indexed constraints, differentiate each index
        expr = c_obj.expr.args[0]
        deriv = differentiate(expr, wrt=var)
        expr = c_obj.expr.args[1]
        deriv -= differentiate(expr, wrt=var)
        jacobian_inequality[cname].append((vname, deriv))
        
# Compute the Jacobian of the objective function with respect to m.origin_model.y
objective_expr = m.origin_model.obj.expr
jacobian_obj = []
for vname in first_names:
    var = m.origin_model.y[vname]
    deriv = differentiate(objective_expr, wrt=var)
    jacobian_obj.append((vname, deriv))


In [8]:
def print_constraints_from_list(model, constraint_names):
    for cname in constraint_names:
        c = model.find_component(cname)
        if c is not None:
            print(f"Constraint: {cname}")
            print(c.expr)
            print("-" * 40)
        else:
            print(f"Constraint '{cname}' not found in model.")

# Example usage:
print_constraints_from_list(m.origin_model, active_inequality_constraints)

Constraint: g_1_con_Demand_constraint1
5.0  <=  x[1,'flow_PG98[1]'] + x[1,'flow_PG98[2]'] + x[1,'flow_PG98[3]'] + x[1,'flow_PG98[4]'] + x[1,'flow_PG98[5]'] + x[1,'flow_PG98[6]'] + x[1,'flow_Import[1]']
----------------------------------------
Constraint: g_1_con_Butane95_constraint
1.7241379310344829*x[1,'flow_ES95[1]'] + 0.039473684210526314*x[1,'flow_Import[2]']  <=  0.05*x[1,volume_ES95]
----------------------------------------
Constraint: g_1_con_Butane98_constraint
1.7241379310344829*x[1,'flow_PG98[1]'] + 0.039473684210526314*x[1,'flow_Import[1]']  <=  0.05*x[1,volume_PG98]
----------------------------------------
Constraint: g_1_con_RON_PG98
0  <=  -98*x[1,volume_PG98] + 132.91139240506328*x[1,'flow_Import[1]'] + 0.0*x[1,'flow_PG98[1]'] + 136.8421052631579*x[1,'flow_PG98[2]'] + 109.23076923076923*x[1,'flow_PG98[3]'] + 123.37662337662337*x[1,'flow_PG98[4]'] + 125.0*x[1,'flow_PG98[5]'] + 124.0*x[1,'flow_PG98[6]'] + 160.34482758620692*x[1,'blin_CDU_LG[2]'] + 159.48275862068968*x[1,'

In [7]:
from pyomo.environ import ConcreteModel, Var

# Create a new Pyomo model
model = ConcreteModel()

# Use the same index and dimension as m.origin_model.y
model.d = Var(m.origin_model.y.index_set())
model.sum_abs_constraint = Constraint(expr=sum(abs(model.d[i]) for i in model.d.index_set()) == 0)

#  check if global solution lies on variable bound
for key in m.origin_model.y:
    var = m.origin_model.y[key]
    lb = var.lb
    ub = var.ub
    val = var.value
    # Check if variable is binary
    if var.is_binary():
        model.add_component(key, Constraint(expr=model.d[key] == 0))
        continue
    if lb is not None and abs(val - lb) < 1e-8:
        model.add_component(f"Lower_Bound_{key}", Constraint(expr=model.d[key] >= 0))
    if ub is not None and abs(val - ub) < 1e-8:
        model.add_component(f"Upper_Bound_{key}", Constraint(expr=model.d[key] <= 0))

# create constraint for each jacobian_equality
for cname, jacobian in jacobian_equality.items():
    
    model.add_component(cname, Constraint(expr=sum(jacobian[i][1] * model.d[jacobian[i][0]] for i in range(len(jacobian))) == 0))
for cname, jacobian in jacobian_inequality.items():
    model.add_component(cname, Constraint(expr=sum(jacobian[i][1] * model.d[jacobian[i][0]] for i in range(len(jacobian))) <= 0))
# Add objective function using jacobian_obj
model.obj = Objective(expr=sum(jacobian_obj[i][1] * model.d[jacobian_obj[i][0]] for i in range(len(jacobian_obj))), sense=minimize)

from pyomo.opt import SolverFactory
solver = SolverFactory('baron')
solver.solve(model, tee=True)

 BARON version 25.3.19. Built: OSX-64 2025-03-19 22:00:43         
 Running on machine Jingzhis-MacBook-Air.local

 BARON is a product of The Optimization Firm.
 For information on BARON, see https://minlp.com/about-baron
 Licensee: Jingzhi Yang at Georgia Institute of Technology, jyang872@gatech.edu.

 If you publish work using this software, please cite publications from
 https://minlp.com/baron-publications, such as: 

 Zhang, Y. and N. V. Sahinidis, Solving continuous and discrete
 nonlinear programs with BARON, Comput Optim Appl (2024). 
 https://doi.org/10.1007/s10589-024-00633-0
 This BARON run may utilize the following subsolver(s)
 For LP/MIP/QP: CLP/CBC                                         
 For NLP: FILTERSQP
 Starting solution is feasible with a value of   0.00000          
 Problem solved during preprocessing
 Lower bound is    0.00000

                         *** Normal completion ***            

 Wall clock time:                     0.01
 Total CPU time used:       

{'Problem': [{'Name': 'problem', 'Lower bound': 0.0, 'Upper bound': 0.0, 'Number of objectives': 1, 'Number of constraints': 497, 'Number of variables': 21, 'Sense': 'unknown', 'Missing bounds': '0', 'Iterations': '-1', 'Node opt': '-2', 'Node memmax': '0', 'Cpu time': 0.0, 'Wall time': 0.01}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Root node time': -100, 'Error rc': 0, 'Time': 0.026131153106689453}], 'Solution': [OrderedDict({'number of solutions': 0, 'number of solutions displayed': 0})]}

In [8]:
model.d.display()

d : Size=20, Index={pickCrude[1], pickCrude[2], pickCrude[3], pickCrude[4], pickCrude[5], pickCrude[6], pickCrude[7], pickCrude[8], pickCrude[9], pickCrude[10], crudeQuantity[1], crudeQuantity[2], crudeQuantity[3], crudeQuantity[4], crudeQuantity[5], crudeQuantity[6], crudeQuantity[7], crudeQuantity[8], crudeQuantity[9], crudeQuantity[10]}
    Key               : Lower : Value : Upper : Fixed : Stale : Domain
    crudeQuantity[10] :  None :   0.0 :  None : False : False :  Reals
     crudeQuantity[1] :  None :   0.0 :  None : False : False :  Reals
     crudeQuantity[2] :  None :   0.0 :  None : False : False :  Reals
     crudeQuantity[3] :  None :   0.0 :  None : False : False :  Reals
     crudeQuantity[4] :  None :   0.0 :  None : False : False :  Reals
     crudeQuantity[5] :  None :   0.0 :  None : False : False :  Reals
     crudeQuantity[6] :  None :   0.0 :  None : False : False :  Reals
     crudeQuantity[7] :  None :   0.0 :  None : False : False :  Reals
     crudeQuantity[

In [9]:
import numpy as np

# Combine all constraint names and their Jacobians
all_jacobians = []
all_constraint_names = []

for cname, jac in jacobian_equality.items():
    all_jacobians.append([coef for _, coef in jac])
    all_constraint_names.append(cname)
# for cname, jac in jacobian_inequality.items():
#     all_jacobians.append([coef for _, coef in jac])
#     all_constraint_names.append(cname)

# Stack into a matrix (rows: constraints, columns: variables in first_names)
A = np.array(all_jacobians)
print("Constraint coefficient matrix shape:", A.shape)

Constraint coefficient matrix shape: (435, 20)


In [10]:
from numpy.linalg import matrix_rank

# Find linearly independent rows of A
def get_independent_rows(A):
    independent_rows = []
    current_rows = []
    for i in range(A.shape[0]):
        test_rows = current_rows + [A[i]]
        if matrix_rank(np.vstack(test_rows)) > len(current_rows):
            independent_rows.append(i)
            current_rows.append(A[i])
    return np.array(independent_rows)

independent_row_indices = get_independent_rows(A)
A_independent = A[independent_row_indices]
print("Shape of A with only independent rows:", A_independent.shape)

Shape of A with only independent rows: (10, 20)
